In [1]:
## 네이버 카페에서 '과외', '과외추천', '과외추천해주세요' 등 검색 
## 지역은 카페명에서 추출해서 "지역별 과외수요"를 알아볼 수 있음 

## https://section.cafe.naver.com/ca-fe/home/search/articles?q=%EA%B3%BC%EC%99%B8%EC%B6%94%EC%B2%9C&p=3

In [2]:
import requests
import json
import pandas as pd 

import os
import sys

from urllib.parse import quote

import re

## Test1..전체 카페에서 "과외추천해주세요" 검색하기
### 문제 : 카페명으로 지역을 어떻게 구분할 것인가? 

In [3]:
client_id = "vD02ixj_z1lpFZ41CeO1"
client_secret = "qk2B9tKmJK"

def cafe_call(keyword,start):
    encText = quote(keyword)
    url = "https://openapi.naver.com/v1/search/cafearticle?query=" + encText+"&sort=date&display=100&start="+str(start) # json 결과
    '''
    요청변수 옵션
    - display : 검색결과 출력건수 (10,100)
    - start : 검색시작위치 (1,1000)
    - sort : 정렬옵션 (sim:유사도순, date:날짜순)
    '''
    result = requests.get(url=url, headers={"X-Naver-Client-Id":client_id,"X-Naver-Client-Secret":client_secret})
    
    if result.status_code == 200:
        #data = json.loads(result.text)
        return result.json()
    else:
        print("error",result.status_code)
    
    '''
    출력결과 옵션
    - lastBuildDate : 검색결과를 생성한 시간 
    - total : 검색결과 문서의 총 개수 (ex : 강남구 과외추천 검색시 2,587)
    - start : 결과문서 중 문서의 시작점-> 한페이지에 100개씩 보여주니까 전체결과가 아님
    - display : 현재 검색결과로 보여준 결과개수 -> items dict안에 들어가는 최대
    - items : 개별검색결과
        - title : 카페 게시글 제목 -> 태그처리필요
        - link : 게시글 링크
        - description : 카페글 요약 -> 태그처리필요
        - cafename : 카페명
        - cafeurl : 카페링크
    
    # 4년치
    
    모든 문서 검색하려고 cafe_call 반복하기?
    # 출처 https://john-analyst.medium.com/%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9D%98-%ED%8A%B9%EC%A0%95-%ED%82%A4%EC%9B%8C%EB%93%9C-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-2c4644001144
    '''

In [4]:
#1000개의 키워드 검색 결과를 받아오기
def get1000results(keyword):
    list = []
    for num in range(0,10):
        list = list + cafe_call(keyword, num * 100 + 1)['items'] # list 안에 키값이 ’item’인 애들만 넣기
    return list

In [5]:
#키워드로 검색한결과
def getDataFrame():
    list = []
    result1 = get1000results("과외추천해주세요")
    result2 = get1000results("과외추천받아요")

    list = list + result1+result2
    
    df = pd.DataFrame.from_records(list)
    df.to_csv("./cafe_data.csv",encoding = 'utf-8-sig')
    
    return df 

In [6]:
# main
df = getDataFrame()
df.head()

,title,link,description,cafename,cafeurl
0,고2수학<b>과외</b>,http://cafe.naver.com/kunsanbeauty/963227,고2여학생 가르쳐주실 수학<b>과외</b>선생님 <b>추천</b> 좀 해<b>주세요...,군산맘 / 미녀맘[비영리카페],https://cafe.naver.com/kunsanbeauty
1,글씨가 너무 악필이에요..ㅠ,http://cafe.naver.com/studymokdong/133191,쪽 지 로 학원이나 <b>과외</b>선생님을 소개받는경우 검증되지않고 단순 홍보인 ...,목동의 공부잘하는 아이,https://cafe.naver.com/studymokdong
2,초등 영어학습지 <b>추천해주세요</b>!,http://cafe.naver.com/tgpia/584228,8시쯤에 오시는 학습지 알아보고있는데 요새는 저희때랑 달리 다양하게 많더라구용~~ ...,"살통영(통영여행,통영가볼만한곳,통영...",https://cafe.naver.com/tgpia
3,[<b>과외</b>]중학 수학 <b>과외</b>,http://cafe.naver.com/bundangchild/14291,스스로 과제를 하기 무척 어려워하여 옆에 끼고 꼼꼼하게 지도해 주실 <b>과외</b...,[ 분당맘스쿨 ] 분당맘 판교맘의 학원...,https://cafe.naver.com/bundangchild
4,(상남동또는성주동) 독서 논술 선생님 <b>추천</b>좀 해<b>주세요</b>,http://cafe.naver.com/masanmam/5701822,----------------------------------------------...,"창원 줌마렐라 [경남,마산,진해,함안,...",https://cafe.naver.com/masanmam


In [7]:
text = "전주 완산구 중앙동 예비 초등 영어<b>과외</b> 풍남동 중등 국어<b>과외</b>"
text2 = re.sub('[<b>|</b>]','',text)
text2

'전주 완산구 중앙동 예비 초등 영어과외 풍남동 중등 국어과외'

In [8]:
## title, description에 <b>,</b> 제거하기 
def clean_letter(x):
    x= re.sub('[<b>|</b>]','',x)
    return x

df['title'] = df['title'].apply(lambda x: re.sub('[<b>|</b>]',' ',x))
df['description'] = df['description'].apply(lambda x: re.sub('[<b>|</b>]',' ',x))

df.head()

,title,link,description,cafename,cafeurl
0,고2수학 과외,http://cafe.naver.com/kunsanbeauty/963227,고2여학생 가르쳐주실 수학 과외 선생님 추천 좀 해 주세요...,군산맘 / 미녀맘[비영리카페],https://cafe.naver.com/kunsanbeauty
1,글씨가 너무 악필이에요..ㅠ,http://cafe.naver.com/studymokdong/133191,쪽 지 로 학원이나 과외 선생님을 소개받는경우 검증되지않고 단순 홍보인 ...,목동의 공부잘하는 아이,https://cafe.naver.com/studymokdong
2,초등 영어학습지 추천해주세요 !,http://cafe.naver.com/tgpia/584228,8시쯤에 오시는 학습지 알아보고있는데 요새는 저희때랑 달리 다양하게 많더라구용~~ ...,"살통영(통영여행,통영가볼만한곳,통영...",https://cafe.naver.com/tgpia
3,[ 과외 ]중학 수학 과외,http://cafe.naver.com/bundangchild/14291,스스로 과제를 하기 무척 어려워하여 옆에 끼고 꼼꼼하게 지도해 주실 과외 ...,[ 분당맘스쿨 ] 분당맘 판교맘의 학원...,https://cafe.naver.com/bundangchild
4,(상남동또는성주동) 독서 논술 선생님 추천 좀 해 주세요,http://cafe.naver.com/masanmam/5701822,----------------------------------------------...,"창원 줌마렐라 [경남,마산,진해,함안,...",https://cafe.naver.com/masanmam


In [9]:
## cafename에 지역구 이름 있는거 찾기-> 새로운컬럼 지역!

gu = ['강서','강동','강남','강북','송도','해남','광명','목동']

In [10]:
df.groupby('cafename').count()

,title,link,description,cafeurl
cafename,,,,
"!▷중정공☞ 중국유학생활,중국여행,무...",1,1,1,1
"""시크먼트"" 명품 정보 커뮤니티 - 샤넬...",2,2,2,2
"""예쁜카페 예카"" 예쁜그릇과앤틱로모노...",5,5,5,5
&lt;오행공&gt; 오산맘들의 행복한 공간,1,1,1,1
&lt;전설의 테너 유채훈 팬카페 : 유채꽃...,1,1,1,1
...,...,...,...,...
화도사랑,1,1,1,1
황인영 영어 카페,12,12,12,12
훈글리시's 토익 독학 공부방,1,1,1,1


## Test2..몇가지 카페에서 '과외추천' 게시글 검색하기?

In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import csv 

def get_cafe_url(url):
    # webdriver 객체 생성
    driver = webdriver.Chrome('./chromedriver')

    # 페이지 접근
    # 날짜지정, 검색어지정, 50개씩보이기
    #url="https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26userDisplay=50%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=1"    #print(url)
    driver.get(url)
    driver.switch_to.frame('cafe_main')

    soup = bs(driver.page_source ,'html.parser')
    soup = soup.find_all(class_ = 'article-board m-tcol-c')[1]# 네이버 카페 구조 확인후 게시글 내용만 가저오기
    #datas = soup.find_all('td', class_ = 'td_article')

    datas_list=[]
    datas = soup.find_all(class_ = 'td_article')
    for data in datas:
        article_title = data.find(class_ = 'article')
        #link = article_title.get('href')
        article_title = article_title.get_text().strip()
        print(article_title)
        datas_list.append(article_title)
        
        #print(article_title)
        #print(baseurl + link)

        #f = open('cafe5.csv', 'a+', newline='', encoding = "euc-kr")# 문자 인코딩 -> euc-kr 형태로 변경하여 사용. 안되면 utf-8로 변경 후 진행
        #wr = csv.writer(f)
        #wr.writerow([article_title])
        #f.close()  
    return datas_list

In [12]:
import pandas as pd
from urllib.parse import urlencode,quote
from datetime import datetime

keyword = "과외추천"
#query=%B0%FA%BF%DC%C3%DF%C3%B5%26
today_date = datetime.today().strftime("%Y-%m-%d")

df_fin = pd.DataFrame()
for pg in range(1,2):
    url = "https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page="+str(pg)
    data_list = get_cafe_url(url)
    
    df = pd.DataFrame(data_list,columns=['article_title'])
    df_fin = pd.concat([df_fin,df],axis=0,ignore_index=True)

#df_fin.head()
filename = today_date+'-cafe_crawling.csv'  
df_fin.to_csv('./'+filename,encoding='utf-8-sig')

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


## Test3..'훈장마을'에서 지역별 채용정보 검색하기 

In [ ]:
!pip uninstall html_table_parser

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import csv 
from html_table_parser import parser_functions as parser

def get_hunjang(url,pg):
    # webdriver 객체 생성
    driver = webdriver.Chrome('./chromedriver')
    
    # 페이지 접근
    driver.get(url)
    soup = bs(driver.page_source ,'html.parser')
    
    soup = soup.find(class_ = 's-adBn-wrap')#채용정보
    if pg <=1 :
        soup = soup.find_all(class_ = 'adRc-inner')[5] #1:스페셜, 2:포커스 3:플러스 4:리스트 5일반
    else:
        soup = soup.find_all(class_ = 'adRc-inner')[0] #1:스페셜, 2:포커스 3:플러스 4:리스트 5일반

    table = soup.find('table', class_='retbl-col')
    rows = table.find_all('tr')
    
    datas_list=[]
    for idx,tr in enumerate(rows):
        result={}
        if idx>0:
            tds = tr.find_all('td')
            result['academy_name'] = tds[1].text.rstrip()
            result['title'] = tds[2].text.strip()
            result['local'] = tds[3].text.rstrip().split("\n")[1]
            result['subject']=tds[3].text.rstrip().split("\n")[2]
            
            result['start_date'] = tds[4].text
            result['end_date'] = tds[5].text
            datas_list.append(result)
                   
    return datas_list
    

In [ ]:
url1 = "https://www.hunjang.com/adopt/adopt_total_list.asp?schSubject=01&schSubject_name=%B1%B9%BE%EE&schSubject=02&schSubject_name=%BF%B5%BE%EE&schSubject=03&schSubject_name=%BC%F6%C7%D0&nPage=1&nPageSize=0&sort_order=&textkey=&textword="
get_hunjang(url1,1)

In [ ]:
url2="https://www.hunjang.com/adopt/adopt_total_list.asp?schSubject=01&schSubject_name=%B1%B9%BE%EE&schSubject=02&schSubject_name=%BF%B5%BE%EE&schSubject=03&schSubject_name=%BC%F6%C7%D0&nPage=2&nPageSize=20&sort_order=&textkey=&textword="
get_hunjang(url2,2)

In [ ]:
import pandas as pd
from urllib.parse import urlencode,quote
from datetime import datetime

today_date = datetime.today().strftime("%Y-%m-%d")

df_fin = pd.DataFrame()
for pg in range(1,31): #마지막페이지30
    url = "https://www.hunjang.com/adopt/adopt_total_list.asp?schSubject=01&schSubject_name=%B1%B9%BE%EE&schSubject=02&schSubject_name=%BF%B5%BE%EE&schSubject=03&schSubject_name=%BC%F6%C7%D0&nPage="+str(pg)+"&nPageSize=0&sort_order=&textkey=&textword="
    data_list = get_hunjang(url,pg)
    
    df = pd.DataFrame(data_list,columns=['academy_name','title','local','subject','start_date','end_date'])
    df_fin = pd.concat([df_fin,df],axis=0,ignore_index=True)

#df_fin.head()
filename = today_date+'-hunjang_crawling.csv'  
df_fin.to_csv('./'+filename,encoding='utf-8-sig')